In [1]:
composer_ip = "synergy.hpedemo.local"

# Welcome to the Synergy Image Streamer POC 
Created by Fredrik Tärnell - 2019-02-20<br>
<br>
## Part 2 - Image Streamer

Due to some missing feature around Image Streamer in the Powershell library at https://github.com/HewlettPackard/POSH-HPOneView
The second part of the demo will be performed using the Python Library which our Ansible modules extends even further.

Jupyter Notebook can be found: https://github.com/frippe75/synergy-jupyter-notebooks
<br><br>

### Requirements
Python + hpOneView Python Library (see: https://github.com/HewlettPackard/python-hpOneView/wiki/HPE-OneView-Python-Windows-Setup-Guide)

### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>


### Additional information

On your HPE OneView appliance, or online<br>
https://{{composer_ip}}/help/cic-rest/en/content/index.html#home.html<br>
https://{{composer_ip}}/api-docs/current/ <br>
http://www.hpe.com/info/oneview/docs <br>
https://developer.hpe.com/

### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

<br><br>
# First section - Simple tasks via Python


### Import the python OneView library with some additional dependencies for this playbook

In [2]:
from hpOneView.oneview_client import OneViewClient
import pandas
from IPython.display import display, Markdown
import pprint as pp

### Configure your environment:

In [3]:
config = {
    "api_version": "600",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "HPEc0nverged!"
    }
}

<br><br>
## Login against the Appliance / Composer instance:
create new object "ov" with the config (see above)

In [4]:
ov = OneViewClient(config)

<br><br>
### The library requires us to add "image_streamer_ip" to the JSON config object
https://github.com/HewlettPackard/python-hpOneView#hpe-synergy-image-streamer

In [5]:
os_deployment_servers_all = ov.os_deployment_servers.get_all(start=0, count=-1, filter='state=Connected')
stream_ip = os_deployment_servers_all[0]['primaryIPV4']
print("\nGet the IP address of the first OS Deployment Server: " + stream_ip)

config['image_streamer_ip'] = stream_ip

print ("\nAdding it to the config dictionary\n")


Get the IP address of the first OS Deployment Server: 10.48.0.105

Adding it to the config dictionary



In [6]:
pp.pprint(config)


{'api_version': '600',
 'credentials': {'authLoginDomain': 'local',
                 'password': 'HPEc0nverged!',
                 'userName': 'Administrator'},
 'image_streamer_ip': '10.48.0.105',
 'ip': 'synergy.hpedemo.local'}


In [7]:
config = {
    "api_version": "600",
    "ip": composer_ip,
    "image_streamer_ip": stream_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "HPEc0nverged!"
    }
}

### Login to Image Streamer
variable "is" cannot be used off course

IMPORTANT: Need to re-login using the new config object and THEN login to image streamer as well

In [8]:
ov = OneViewClient(config)
streamer = ov.create_image_streamer_client()

<br><br>
## Uploading Image Streamer artifacts
<br>
The artifacts are pulled down from our github repo which are release specific to
<br>

https://github.com/HewlettPackard/image-streamer-esxi/tree/v4.1/artifact-bundles<br>
https://github.com/HewlettPackard/image-streamer-rhel/tree/V4.1/artifact-bundles<br>

### Start by downloading the artifacts from our github repo

In [19]:
import requests
import os.path

#https://github.com/HewlettPackard/image-streamer-esxi/raw/v4.2/artifact-bundles/
esx_file = 'HPE - ESXi -2018-07-31-v4.2.zip'
rhel_file = 'HPE - RHEL-2018-09-11-v4.2.zip'


if not os.path.exists(esx_file):
    print("File " + esx_file + " does not exist going to download it")
    url = 'https://github.com/HewlettPackard/image-streamer-esxi/raw/v4.2/artifact-bundles/' + esx_file
    r = requests.get(url, allow_redirects=True)
    open(esx_file, 'wb').write(r.content)


if not os.path.exists(rhel_file):
    print("File " + rhel_file + " does not exist going to download it")
    url = 'https://github.com/HewlettPackard/image-streamer-rhel/raw/v4.2/artifact-bundles/' + rhel_file
    r = requests.get(url, allow_redirects=True)
    open(rhel_file, 'wb').write(r.content)

if os.path.exists(esx_file):
    print (esx_file + " downloaded successfully!")
if os.path.exists(rhel_file):
    print (rhel_file + " downloaded successfully!")

File HPE - ESXi -2018-07-31-v4.2.zip does not exist going to download it
File HPE - RHEL-2018-09-11-v4.2.zip does not exist going to download it
HPE - ESXi -2018-07-31-v4.2.zip downloaded successfully!
HPE - RHEL-2018-09-11-v4.2.zip downloaded successfully!


### Get a list of currently available artifacts bundles, ESX should not be one of them

In [21]:
# Get all Artifacts Bundle
print("\nGet all Artifact Bundles:")

artifact_bundles = streamer.artifact_bundles.get_all()

for artifacts_bundle in artifact_bundles:
    print(" - " + artifacts_bundle['name'])
    


Get all Artifact Bundles:
 - HPE - ESXi -2018-07-31-v4.2
 - HPE - RHEL-2018-09-11-v4.2
 - HPE_Foundation_Artifacts-4_20
 - HPE_Support_Artifacts-4_20


<br><br>
### Uploading the ESXi Artifacts 4.10 from file

Python client documentation
https://hewlettpackard.github.io/python-hpOneView/hpOneView.image_streamer.resources.html?highlight=upload#hpOneView.image_streamer.resources.artifact_bundles.ArtifactBundles.upload_bundle_from_file

In [20]:
# Upload an Artifact Bundle from file
print("\nUpload an Artifact Bundle from file:")

print("\nDescription:")
esx_bundle = streamer.artifact_bundles.upload_bundle_from_file(esx_file)
print(esx_bundle['description']) 

print("\nSuccesfully added ESXi!")


Upload an Artifact Bundle from file:

Description:
ImageStreamer artifacts for ESXi 5.x and ESXi 6.x till 6.5. (c) Copyright 2018 Hewlett Packard Enterprise Development LP. Licensed under the Apache License, Version 2.0 (the "License");you may not use this file except in compliance with the License

Succesfully added ESXi!


<br><br>
### Uploading the RHEL Artifacts 4.10 from file


In [17]:
# Upload an Artifact Bundle from file
print("\nUpload an Artifact Bundle from file:")

print("\nDescription:")
rhel_bundle = streamer.artifact_bundles.upload_bundle_from_file(rhel_file)
print(rhel_bundle['description']) 

print("\nSuccesfully added RHEL!")


Upload an Artifact Bundle from file:

Description:
ImageStreamer artifacts for RHEL 7.X personalization and generalization.(c) Copyright 2018 Hewlett Packard Enterprise Development LP. Licensed under the Apache License, Version 2.0 (the "License")

Succesfully added RHEL!


<br><br>
### Extract the newly added Artifacts bundles  

In [ ]:
# Add both bundles to an array (not nessesary)
# Placed RHEL last since it for some reason hangs waiting for the task to complete. Give it 2-3 minutes and then interrupt
# the Jupyter kernel in the menu
bundles = [esx_bundle, rhel_bundle]

# Extract each Artifact Bundle
print("\nExtracting the Artifact Bundles:")
for bundle in bundles:
    print(" - " + bundle['name'])
    response = streamer.artifact_bundles.extract_bundle(bundle)
    #print(response)


Extracting the Artifact Bundles:
 - HPE - ESXi -2018-07-31-v4.2
 - HPE - RHEL-2018-09-11-v4.2


<br><br>
### Upload a pre-made ESXi Golden Image 6.5u2
<blink>
    Golden image zip file has to be copied into the notebook directory of the Docker container at this stage
</blink>

In [24]:
esx_golden_file = 'VMware-ESXi-6.5.0-Update2-8294253-HPE-Gen9plus-650.U2.10.3.0.24-Sept2018.zip'

golden_image_upload = {
    "name": "ESXi-65u2",
    "description": "Pre-made ESXi 6.5u2 image part of the HPE POC - TO-BE-DELETED",
}

# Upload a Golden Image
print("Uploading the Golden Image from " + esx_golden_file)
golden_esx_image = streamer.golden_images.upload(esx_golden_file, golden_image_upload)
print(golden_esx_image)
print("***** done *****\n")

Uploading the Golden Image from VMware-ESXi-6.5.0-Update2-8294253-HPE-Gen9plus-650.U2.10.3.0.24-Sept2018.zip
{'type': 'GoldenImage', 'uri': '/rest/golden-images/e6554cf3-fb06-4ee5-b1c1-24a57999ed8b', 'category': 'golden-images', 'eTag': '2019-03-04T14:51:31.193Z', 'created': '2019-03-04T14:51:31.193Z', 'modified': '2019-03-04T14:51:31.193Z', 'id': 'e6554cf3-fb06-4ee5-b1c1-24a57999ed8b', 'name': 'ESXi-65u2', 'description': 'Pre-made ESXi 6.5u2 image part of the HPE POC - TO-BE-DELETED', 'status': 'OK', 'size': 380839141, 'checkSum': '1c2e137bebee8496db5c7dc72c04bab139e9eed298470330a609f841d6b8bc3e', 'readOnly': False, 'importedFromBundle': False, 'bundleName': None, 'bundleURI': None, 'artifactBundleCategory': None, 'imageCapture': False, 'osVolumeURI': None, 'osVolumeName': None, 'osVolumeCategory': None, 'buildPlanUri': None, 'buildPlanName': None, 'buildPlanCategory': None, 'dependentArtifacts': None}
***** done *****



In [25]:
# DEBUG Cell - TODO: Remove at some point
pp.pprint(esx_bundle)

{'artifactsCount': 15,
 'artifactsbundleID': '94d2ab8d-5bbd-48af-a8f4-446b43cc98c7',
 'backupService': False,
 'buildPlans': [{'bpID': '24c9483c-5c88-491b-b56a-630f20b57ce2',
                 'buildPlanName': 'HPE - ESXi - deploy with multiple '
                                  'management NIC HA config- 2018-07-31',
                 'description': 'Deploy ESXi 5.x - 6.5 in a multi-frame '
                                'environment containing a pair of '
                                'ImageStreamer appliances. This buildplan '
                                'configures HA for iSCSI boot connections to '
                                'ESXi volume. (c) Copyright 2018 Hewlett '
                                'Packard Enterprise Development LP. Licensed '
                                'under the Apache License, Version 2.0  (the '
                                '"License");...',
                 'planScriptName': 'HPE - ESXi - umount - 2017-03-15,HPE - '
                       

In [ ]:
# Get all Deployment Plans
print("\nGet all Deployment Plans")
deployment_plans = streamer.deployment_plans.get_all()
for deployment_plan in deployment_plans:
    print(deployment_plan['name'])

In [ ]:
# The name of the one to use
#esx_bp_name = 'HPE - ESXi - deploy with multiple management NIC HA config- 2018-07-31'
esx_bp_name = 'HPE- ESXi - deploy in single frame non-HA config- 2018-07-31'

# Get all Build Plans
print('\nGet the URI of "' + esx_bp_name +'"')

# get_by returns an array thereof the [0]
esx_bp = streamer.build_plans.get_by('name', esx_bp_name)
esx_bp_uri = esx_bp[0]['uri']
print(esx_bp_uri)

In [83]:
# Build Plan URI was pulled from esx_bundle
dp_information = {
    "name": "ESX 65u2 POC Deployment Plan",
    "description": "Pre-made ESX 65u2 POC Deployment Plan for the HPE POC - TO-BE-DELETED",
    "hpProvided": "false",
    "type": "OEDeploymentPlanV5",
    "oeBuildPlanURI": esx_bp_uri
}
pp.pprint(dp_information)

# Create a Deployment Plan
print("Create a Deployment Plan")
deployment_plan = streamer.deployment_plans.create(dp_information)
pp.pprint(deployment_plan)

{'description': 'Pre-made ESX 65u2 POC Deployment Plan for the HPE POC - '
                'TO-BE-DELETED',
 'hpProvided': 'false',
 'name': 'ESX 65u2 POC Deployment Plan',
 'oeBuildPlanURI': '/rest/build-plans/ad392ee8-1639-48ca-8fea-6a8b3501c055',
 'type': 'OEDeploymentPlanV5'}
Create a Deployment Plan


HPOneViewException: ('Error while adding the deployment plan.', {'errorCode': 'OEDP_CREATE_CA_MISMATCH_ERROR', 'message': 'Error while adding the deployment plan.', 'details': 'Unable to add the deployment plan. There is a mismatch between custom attributes of the deployment plan and custom attributes of the OS build plan used by this deployment plan.', 'recommendedActions': ['The custom attribute list of the deployment plan has to match exactly with the list of the custom attributes of OS build plan used by this deployment plan. Verify the custom attributes and try again. '], 'errorSource': None, 'nestedErrors': [], 'data': {}})

In [40]:
# Get the Artifacts Bundle by Name
print("\nGet Artifact Bundles by Name")
artifact_bundle = streamer.artifact_bundles.get_by_name(artifact_bundles_to_be_created['name'])
pprint(artifact_bundle)


Get Artifact Bundles by Name


NameError: name 'artifact_bundles_to_be_created' is not defined

In [50]:
# Get the Deployment Plan by name
print("\nGet the Deployment Plan by name")
deployment_plan_by_name = streamer.deployment_plans.get_by('name', '22044547-6f5e-4cbc-af6b-197273d9f32c')
pp.pprint(deployment_plan_by_name)


Get the Deployment Plan by name
[]
